In [ ]:
하이퍼파라미터 최적화를 할 때 검증 세트에 overfitting 된다는 것을 주의해야 한다.

In [ ]:
Batch Normalization

이전 예제는 모델에 데이터를 주입하기 전에 정규화 했다.
하지만 데이터 정규화는 네트워크에서 일어나는 모든 변환 후에도 고려되어야 한다.
Dense나 Conv2D 층에 들어가는 데이터의 평균이 0이고 분산이 1이더라도 출력되는 데이터가 동일한 분포를 가질 것이라 기대하기 어렵다.

배치 정규화(Batch Normalization)는 훈련하는 동안 평균과 분산이 바뀌더라도 이에 적응하여 데이터를 정규화한다.
훈련 과정에 사용된 배치 데이터의 평균과 분산에 대한 지수 이동 평균(exponential moving average)을 내부에 유지한다.
* 지수 이동 평균 : v = v * momentum + v_new * (1 - momentum)으로 계산

배치 정규화의 주요 효과는 잔차 연결과 매우 흡사하게 그래디언트의 전파를 도와준다.
(입력에 비하여 활성화 함수의 출력이 너무 작거나 커지면 변화율이 급격히 작아져 역전파되는 그래디언트도 매우 줄어들게 된다.)

ex) BatchNormalization

conv_model.add(layers.Conv2D(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())

dense_model.add(layers.Dense(32, activation = 'relu'))
dense_model.add(layers.BatchNormalization())

BatchNormalization 클래스에는 정규화 할 특성 축을 지정하는 axis 매개변수가 있다. 이 매개변수의 기본값은 입력 텐서의 마지막 축을 나타내는 -1.
data_format을 "channels_last"로 하여 Dense, Conv1D, RNN, Conv2D 층을 사용할 때는 맞는 값이다.
하지만 data_format을 "channels_first"로 사용하는 경우에는 특성 축이 1이다.(0번째 축은 배치 차원) 이때 axis 매개변수 값은 1이다.

In [ ]:
Depthwise Separable Convolution(깊이별 분리 합성곱)

SeparableConv2D / 이 층은 입력 채널별로 따로따로 공간 방향의 합성곱을 수행한다. 그 다음 점별 합성곱(1*1 합성곱)을 통해 출력 채널을 합친다.
이는 공간 특성의 학습과 채널 방향 특성의 학습을 분리하는 효과를 낸다.
입력에서 공간상 위치는 상관관계가 크지만 채널별로는 매우 독립적이라고 가정한다면 타당하다.

이 방법은 모델 파라미터와 연산의 수를 크게 줄여 주기 때문에 작고 더 빠른 모델을 만든다. 합성곱을 통해 더 효율적으로 표현을 학습하기 때문에 적은 
데이터로도 더 좋은 표현을 학습하고, 결국 성능이 더 높은 모델을 만든다.

이 장점은 제한된 데이터로 작은 모델을 처음부터 훈련시킬 때 특히 더 중요하다.

작은 데이터셋에서 이미지 분류 문제(softmax classification)를 위한 가벼운 깊이별 분리 컨브넷을 만드는 예시

import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential, Model
from keras import layers

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()

model.add(layers.SeparableConv2D(32, 3, activation = 'relu', input_shape(height, width, channels,)))
model.add(layers.SeparableConv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation = 'relu'))
model.add(layers.SeparableConv2D(128, 3, activation = 'relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation = 'relu'))
model.add(layers.SeparableConv2D(128, 3, activation = 'relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(num_classes, activation = 'softmax'))

model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy')

In [ ]:
Hyperparameter Optimization (하이퍼파라미터 최적화)

ex) Bayesian Optimization(베이지안 최적화), Genetic Algorithms(유전 알고리즘), Random Search(랜덤 탐색)


In [ ]:
Model Ensemble(모델 앙상블)
각자의 가정(고유한 모델 구조와 랜덤 가중치 초기화)을 이용하고 각자의 관점으로 이해한다.

분류를 예시로 들어보면 분류기 예측을 (앙상블하기 위해)합치는 가장 쉬운 방법은 추론할 때 나온 예측을 평균 내는 것.

preds_a = model_a.predict(x_val)
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_c.predict(x_val)

final_preds = 0.25 * (preds_a + preds_b + preds_c + preds_d)

이 방식은 분류기들이 어느 정도 비슷하게 좋게 작동하는 경우 잘 작동한다. 분류기 중 하나가 다른 모델보다 월등히 나쁜 경우 최종 예측은 앙상블에 있는
가장 좋은 분류기 만큼 좋지 않을 수 있다.

분류기를 앙상블하는 좋은 방법중 하나는 검증 데이터에서 학습된 가중치를 사용하여 가중 평균하는 것이다.
위 final_preds에서 0.25가 아닌 각기 다른 가중치를 곱해준다.

